# Exercise03 : Web サービスとして公開する

最後にモデルを Web サービスとして公開します。

このコードを実行する前に、**[Exercise02 : リモート CPU VM でのトレーニング](./exercise02_train_on_remote.ipynb) でモデルの登録を完了させてください**。


## スコアリング スクリプトの作成 (.py)

ウェブサービスとしてデプロイするために、まず以下のようなスコアリングコードを生成します。<br>
AML のこのエントリスクリプトは、 ``init()`` と ``run()`` の両方を含む必要があります。

> 注意 : サービングコンピュート(VM)は [Managed Identity Endpoint](https://docs.microsoft.com/azure/active-directory/managed-identities-azure-resources/how-to-use-vm-token) を提供します。
(スクリプトは、システム割り当てアイデンティティとユーザ割り当てアイデンティティの両方を使用できます)。スクリプトは、安全な情報を提供することなく、Azure リソースのアクセス許可を取得することができます。


In [ ]:
%%writefile scripts/score.py
import os
import json
import logging
from mlflow.pyfunc import load_model

# Called when the service is loaded
def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # Please provide your model's folder name if there is one
    logging.info("AZUREML_MODEL_DIR: " + os.environ["AZUREML_MODEL_DIR"])

    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "models")
    model = load_model(model_path)  
    logging.info("Init complete")

def run(mini_batch):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info(f"run method start: {__file__}, run({mini_batch})")

    input = json.loads(mini_batch)["data"]
    logging.info(f"input: {input}")

    predictions = model.predict(input)
    logging.info('Predictions:' + str(predictions))
    logging.info("Request processed")

    return predictions.tolist() # return a dataframe or a list

## マネージドエンドポイントの作成

マネージド オンライン エンドポイントでは、デプロイ トポロジーに **エンドポイント** と **デプロイ** が存在します。<br>
1 つのエンドポイントで複数のデプロイを実行し、これらの複数のデプロイに適切なトラフィックを割り当てることができます。


まず、デプロイ先としてマネージドエンドポイントを作成します。<br>
ここで、**``name`` はユニークでなければならないことに注意して、任意のユニークネーム** を指定します。


In [ ]:
endpoint_name = "{UNIQUE_ENDPOINT_NAME}"

さらに以下の ``{UNIQUE_ENDPOINT_NAME}`` を置き換えてください。

In [ ]:
%%writefile 03_managed_endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: {UNIQUE_ENDPOINT_NAME}
auth_mode: key

In [ ]:
!az ml online-endpoint create --file 03_managed_endpoint.yml

ML Studio の左メニュー「エンドポイント」をクリックすると実際にエンドポイントが登録されていることが確認できます。

## Web サービスとしてデプロイする
次にサービング用のコード (`score.py`) を先のエンドポイントに Web サービスとしてデプロイします。

デプロイ前に、サービング環境用の conda 設定を作成します。`azureml-defaults` を含める必要があります。

In [ ]:
%%writefile environments/03_conda_env.yml
name: serving_example
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.8.5
  - pip
  - pip:
      - azureml-defaults
      - azureml-mlflow==1.41.0
      - scikit-learn==1.0.2
      - pandas==1.1.5
      - joblib==1.0.0
      - matplotlib==3.3.3

デプロイ用の yaml 設定をして Web サービスをデプロイします。

マネージドエンドポイントのモデル（またはコード）を変更した場合、複数のデプロイメントを送信し、トラフィック割り当てを転送しても、混乱は生じません。
以下の ``--all-traffic`` オプションを指定すると、すべてのトラフィック (100% トラフィック) がこの単一のデプロイメントに割り当てられます。

この例では、Exercise04 で学習したモデルを使用するため、**このコードを実行する前に、"[Exercise02 : リモート CPU VM でのトレーニング](./exercise02_train_on_remote.ipynb)" を実行します**。

以下の ``{UNIQUE_ENDPOINT_NAME}`` を置き換えてください。

> 注意 : 以下の ``instance_count`` を増やすことで、計算をスケールすることができます。(オートスケールの設定も可能です。)


In [ ]:
%%writefile 03_managed_deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: {UNIQUE_ENDPOINT_NAME}
model: azureml:diabetes_model_oh4ml@latest
code_configuration:
  code: ./scripts
  scoring_script: score.py
environment: 
  conda_file: ./environments/03_conda_env.yml
  image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
instance_type: Standard_DS2_v2
instance_count: 1

### デプロイ実行

デプロイの完了まで、10 分程度かかります。ML Studio の左メニュー「エンドポイント」をクリックし、エンドポイントの詳細を開くとデプロイの状況について確認できます。

In [ ]:
!az ml online-deployment create --file 03_managed_deployment.yml \
  --all-traffic

デプロイエラーが発生した場合、以下のような方法でログを取得できます。


In [ ]:
!az ml online-deployment get-logs \
  --endpoint-name $endpoint_name \
  --name blue

> 注意 : クラウド上にデプロイを送信する前に、ローカルの Docker ランタイム上でデプロイし、デバッグすることができます。([こちら](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints)を参照してください)<br>
> Visual Studio Code では、ローカルのデプロイにデバッガーをアタッチすることもできます。

## Web サービスをテストする

デプロイした Web サービスを呼び出して、返された結果を Python で確認してみましょう。

まず、デプロイした Web サービスの URI（アドレス）を確認します。

In [ ]:
!az ml online-endpoint show \
  --name $endpoint_name \
  --query scoring_uri

このエンドポイント用のキークレデンシャルを抽出します。

In [ ]:
!az ml online-endpoint get-credentials \
  --name $endpoint_name

それでは、Python でスコアリング Web サービスを呼び出してみましょう。<br>
(** 以下の ``ENDPOINT_URI`` と ``API_KEY`` を自分のものに置き換えてください**)


In [ ]:
import requests
import json

SERVING_URI = "{ENDPOINT_URI}"
API_KEY = "{API_KEY}"

input = [[0.0380759064334241,0.0506801187398187,0.0616962065186885,0.0218723549949558,-0.0442234984244464,-0.0348207628376986,-0.0434008456520269,-0.00259226199818282,0.0199084208763183,-0.0176461251598052]]

# Invoke web service !
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ API_KEY)
} 

values = json.dumps(input)
input_data = "{\"data\": " + values + "}"
print(input_data)
http_res = requests.post(
    SERVING_URI,
    input_data,
    headers = headers)
print('Predicted : ', http_res.text)


## エンドポイントの削除
エンドポイントが不要になった場合、以下のようにして削除できます。

In [ ]:
#!az ml online-endpoint delete \
#  --name $endpoint_name \
#  --yes